In [2]:
import pandas as pd
import numpy as np


In [3]:
books=pd.read_csv('/mnt/e/Book-Recommender/Books.csv',low_memory=False)
ratings=pd.read_csv('/mnt/e/Book-Recommender/Ratings.csv')
users=pd.read_csv('/mnt/e/Book-Recommender/Users.csv')

In [4]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 8)
(278858, 3)
(1149780, 3)


In [5]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


# Popularity Based

conditions to popularity Based recommender 

top 50 only
Rating <250


In [6]:
booksWtRatings=ratings.merge(books,on="ISBN")


we have to group the same book and take the max number of ratings

In [7]:
num_Rating_df=booksWtRatings.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_Rating_df.rename(columns={'Book-Rating':'No_of_Rating'},inplace=True)
num_Rating_df

,Book-Title,No_of_Rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


now making a new df for avg rating

In [8]:
avg_Rating_df=booksWtRatings.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_Rating_df.rename(columns={'Book-Rating':'AvgRating'},inplace=True)
avg_Rating_df

/tmp/ipykernel_273/3902616928.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_Rating_df=booksWtRatings.groupby('Book-Title').mean()['Book-Rating'].reset_index()


,Book-Title,AvgRating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [9]:
popular_df=num_Rating_df.merge(avg_Rating_df,on="Book-Title")
popular_df.head()

,Book-Title,No_of_Rating,AvgRating
0,A Light in the Storm: The Civil War Diary of ...,4,2.25
1,Always Have Popsicles,1,0.00
2,Apple Magic (The Collector's series),1,0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.00
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.00


In [10]:
popular_df=popular_df[popular_df['No_of_Rating']>=250].sort_values('AvgRating',ascending=False).head(50)
popular_df.shape


(50, 3)

In [11]:
popular_df=popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['ISBN','Book-Title','Book-Author','Image-URL-M','No_of_Rating','AvgRating']]


# collabrative filtering recommender System

step 1 make a df whith users and books they have rated + how much they have rated min:200 
Books must have more that 50 rating to qualify for recommendation 

In [12]:
x=booksWtRatings.groupby('User-ID').count()['Book-Rating']>200
topRatingUsers=x[x].index

In [13]:
filtered_rating_df=booksWtRatings[booksWtRatings['User-ID'].isin(topRatingUsers)]

In [14]:
y=filtered_rating_df.groupby('Book-Title').count()['Book-Rating']>=50
famous_books=y[y].index

In [15]:
final_df=filtered_rating_df[filtered_rating_df['Book-Title'].isin(famous_books)]

In [16]:
pt=final_df.pivot_table(index='Book-Title',columns='User-ID')

/tmp/ipykernel_273/1334689832.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pt=final_df.pivot_table(index='Book-Title',columns='User-ID')


In [17]:
pt.fillna(0,inplace=True)


In [18]:
pt

Book-Rating                \
User-ID                                                 254    2276   2766     
Book-Title                                                                     
1984                                                       9.0    0.0    0.0   
1st to Die: A Novel                                        0.0    0.0    0.0   
2nd Chance                                                 0.0   10.0    0.0   
4 Blondes                                                  0.0    0.0    0.0   
A Bend in the Road                                         0.0    0.0    7.0   
...                                                        ...    ...    ...   
Year of Wonders                                            0.0    0.0    0.0   
You Belong To Me                                           0.0    0.0    0.0   
Zen and the Art of Motorcycle Maintenance: An I...         0.0    0.0    0.0   
Zoya                                                       0.0    0.0    0.0   
\O\" Is for Outlaw"                                        0.0    0.0    0.0   

                                                                         \
User-ID                                            2977   3363   4017     
Book-Title                                                                
1984                                                  0.0    0.0    0.0   
1st to Die: A Novel                                   0.0    0.0    0.0   
2nd Chance                                            0.0    0.0    0.0   
4 Blondes                                             0.0    0.0    0.0   
A Bend in the Road                                    0.0    0.0    0.0   
...                                                   ...    ...    ...   
Year of Wonders                                       7.0    0.0    0.0   
You Belong To Me                                      0.0    0.0    0.0   
Zen and the Art of Motorcycle Maintenance: An I...    0.0    0.0    0.0   
Zoya                                                  0.0    0.0    0.0   
\O\" Is for Outlaw"                                   0.0    0.0    0.0   

                                                                         \
User-ID                                            4385   6251   6323     
Book-Title                                                                
1984                                                  0.0    0.0    0.0   
1st to Die: A Novel                                   0.0    0.0    0.0   
2nd Chance                                            0.0    0.0    0.0   
4 Blondes                                             0.0    0.0    0.0   
A Bend in the Road                                    0.0    0.0    0.0   
...                                                   ...    ...    ...   
Year of Wonders                                       0.0    0.0    0.0   
You Belong To Me                                      0.0    0.0    0.0   
Zen and the Art of Motorcycle Maintenance: An I...    0.0    0.0    0.0   
Zoya                                                  0.0    0.0    0.0   
\O\" Is for Outlaw"                                   0.0    0.0    0.0   

                                                           ...                \
User-ID                                            6543    ... 271705 273979   
Book-Title                                                 ...                 
1984                                                  0.0  ...   10.0    0.0   
1st to Die: A Novel                                   9.0  ...    0.0    0.0   
2nd Chance                                            0.0  ...    0.0    0.0   
4 Blondes                                             0.0  ...    0.0    0.0   
A Bend in the Road                                    0.0  ...    0.0    0.0   
...                                                   ...  ...    ...    ...   
Year of Wonders                                       0.0  ...    0.0    9.0   
You Belong To Me            

In [19]:
from sklearn.metrics.pairwise import cosine_similarity 
similarity_Score=cosine_similarity(pt)


In [54]:
def recommend(Book_name):
    index=np.where(pt.index==Book_name)[0][0]
    similar_items=sorted(list(enumerate(similarity_Score[index])),key=lambda x:x[1],reverse=True)[1:9]
    data=[]
    for i in similar_items:
        items=[]
        temp_df=books[books['Book-Title']== pt.index[i[0]]]
        items.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        items.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        items.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-S'].values))
        
        data.append(items)
        
    return data
        
        
        
        

In [55]:
recommend('1984')

[['Animal Farm',
  'George Orwell',
  'http://images.amazon.com/images/P/0451526341.01.THUMBZZZ.jpg'],
 ["The Handmaid's Tale",
  'Margaret Atwood',
  'http://images.amazon.com/images/P/0449212602.01.THUMBZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.THUMBZZZ.jpg'],
 ['The Vampire Lestat (Vampire Chronicles, Book II)',
  'ANNE RICE',
  'http://images.amazon.com/images/P/0345313860.01.THUMBZZZ.jpg'],
 ['The Hours : A Novel',
  'Michael Cunningham',
  'http://images.amazon.com/images/P/0312243022.01.THUMBZZZ.jpg'],
 ['Fahrenheit 451',
  'Ray Bradbury',
  'http://images.amazon.com/images/P/3257208626.01.THUMBZZZ.jpg'],
 ['The Catcher in the Rye',
  'J.D. Salinger',
  'http://images.amazon.com/images/P/0316769487.01.THUMBZZZ.jpg'],
 ['Naked',
  'David Sedaris',
  'http://images.amazon.com/images/P/0316777730.01.THUMBZZZ.jpg']]

In [22]:
pt.index[424]


'Secrets'

In [62]:
popular_df

,ISBN,Book-Title,Book-Author,Image-URL-M,No_of_Rating,AvgRating
0,0439136350,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
3,0439139597,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
5,0590353403,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
9,043935806X,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
13,0439064872,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
16,0345339681,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
17,0345339703,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,368,4.948370
26,059035342X,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.895652
28,0345339711,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...,260,4.880769
39,0446310786,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,510,4.700000


In [56]:
import pickle
#pickle.dump(popular_df,open('popularBooks','wb'))

In [58]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_Score,open('similarity_Score.pkl','wb'))

In [64]:
books['Image-URL-S'][0]


'http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg'